# Homework 1

## Question 1

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [2]:
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

len(df.columns)

19

## Question 2

In [3]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.std()

np.float64(42.59435124195458)

## Question 3

In [4]:
len_total = len(df.index)

df = df[(df.duration >= 1) & (df.duration <= 60)]

len_filtered = len(df.index)

f'fraction left = {np.round(100*len_filtered/len_total)}%'

'fraction left = 98.0%'

## Question 4

In [5]:
features = ['PULocationID', 'DOLocationID']

df[features] = df[features].astype(str)

train_dicts = df[features].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018346 stored elements and shape (3009173, 515)>

## Question 5

In [6]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261946248764

## Question 6

In [7]:
# Prepare test dataset
df = pd.read_parquet('data/yellow_tripdata_2023-02.parquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

features = ['PULocationID', 'DOLocationID']
df[features] = df[features].astype(str)
test_dicts = df[features].to_dict(orient='records')
X_test = dv.transform(test_dicts)

target = 'duration'
y_test = df[target].values

y_pred = lr.predict(X_test)

root_mean_squared_error(y_test, y_pred)

7.811817544074326

In [8]:
from pathlib import Path
import pickle

folder = Path('./artifacts')
folder.mkdir(parents=True, exist_ok=True)

with open('artifacts/dv.bin', 'wb') as f:
    pickle.dump(dv, f)

with open('artifacts/model.bin', 'wb') as f:
    pickle.dump(lr, f)